In [2]:
import pandas as pd
import csv
import os
import numpy as np
import datetime
import time
Working_dir = os.getcwd()

In [41]:
zabbix_df = pd.read_excel( os.path.join( Working_dir, "zabbix_month_py.xlsx" ) )
lsf_df = pd.read_excel( os.path.join( Working_dir, "lsf_month_py.xlsx" ) )

In [3]:
col_names = ['timestamp_py','date_py','time_py','apps_running','apps_running_ids',\
             'apps_started_this_hour', 'apps_ended_this_hour',\
             'apps_ended_success_this_hour', 'apps_ended_exit_this_hour']
apps_at_time_df = pd.DataFrame(np.zeros((zabbix_df.shape[0], len(col_names) )), \
                             columns = col_names )
apps_at_time_df.timestamp_py = zabbix_df["Start_Timestamp (1h interval)"].apply(lambda x: \
                                                    datetime.datetime.utcfromtimestamp(x))
apps_at_time_df.date_py = zabbix_df.date_py.copy()
apps_at_time_df.time_py = zabbix_df.time_py.copy()
apps_at_time_df['power_avg']=zabbix_df['avg_power'].copy()
apps_at_time_df['power_min']=zabbix_df['min_power'].copy()
apps_at_time_df['power_max']=zabbix_df['max_power'].copy()
apps_at_time_df['energy_avg (kWh)']=zabbix_df['avg_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.
apps_at_time_df['energy_min (kWh)']=zabbix_df['min_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.
apps_at_time_df['energy_max (kWh)']=zabbix_df['max_power'].copy()*zabbix_df['measure time in seconds'].copy()/60000.
apps_at_time_df.head()

,timestamp_py,date_py,time_py,apps_running,apps_running_ids,apps_started_this_hour,apps_ended_this_hour,apps_ended_success_this_hour,apps_ended_exit_this_hour,power_avg,power_min,power_max,energy_avg (kWh),energy_min (kWh),energy_max (kWh)
0,2017-02-19 12:00:00,2017-02-19,12:00:00,0.0,0.0,0.0,0.0,0.0,0.0,40637,38600,41500,40.637000,38.600000,41.500000
1,2017-02-19 13:00:00,2017-02-19,13:00:00,0.0,0.0,0.0,0.0,0.0,0.0,40863,40010,41530,40.863000,40.010000,41.530000
2,2017-02-19 14:00:00,2017-02-19,14:00:00,0.0,0.0,0.0,0.0,0.0,0.0,39971,37740,41430,39.804454,37.582750,41.257375
3,2017-02-19 15:00:00,2017-02-19,15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,38517,36190,39470,38.677487,36.340792,39.634458
4,2017-02-19 16:00:00,2017-02-19,16:00:00,0.0,0.0,0.0,0.0,0.0,0.0,38735,38330,39200,38.735000,38.330000,39.200000


In [4]:
lsf_current_line = lsf_df.iloc[5]
# lsf_current_line.start_time_py.hour
lsf_df_start_time = datetime.datetime.utcfromtimestamp(lsf_current_line["start unixtime"])
sum(1 for index, row in lsf_df.iterrows() if row['job status'].lower()=="done" or\
    row['job status'].lower()=="exit") == lsf_df.shape[0]
# print(datetime.datetime(lsf_df_start_time.date(), lsf_df_start_time.time().hour,0,0))


True

In [5]:
apps_at_time_df['apps_running'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['apps_started_this_hour'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['apps_ended_this_hour'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['apps_ended_exit_this_hour'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['apps_ended_success_this_hour'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['cores_sum'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['cores_avg'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['cores_min'] = lsf_df['number of cores used'].max()
apps_at_time_df['cores_max'] = lsf_df['number of cores used'].min()

apps_first_time_moment = apps_at_time_df.timestamp_py.iloc[0]
apps_last_time_moment = apps_at_time_df.timestamp_py.iloc[apps_at_time_df.shape[0]-1]
i = 0
for index, lsf_current_line in (lsf_df.iterrows()):
    i+=1
    cores_used = lsf_current_line["number of cores used"]
    #find first line of date and start_time (NOT+1 but ==) in apps_at_time_df 
    #which are the same as in lsf
    #add +1 process during the reported hour
    #go through time in apps_df 
    #while date<=end_date and end_time.hour (in lsf_df) (NOT+1 but ==) <= time in apps_df
    
    lsf_df_start_time = datetime.datetime.utcfromtimestamp(lsf_current_line["start unixtime"])
    lsf_df_end_time = datetime.datetime.utcfromtimestamp(lsf_current_line["stop unixtime"])
    
    if (lsf_df_start_time >= apps_first_time_moment):
        apps_df_current_line_no = apps_at_time_df[ (apps_at_time_df.date_py == \
                                                    lsf_df_start_time.date()) \
                                                 & (apps_at_time_df.time_py == \
                                                    datetime.time( lsf_df_start_time.time().hour,0,0))].index[0]
    elif (lsf_df_end_time >= apps_first_time_moment):
          apps_df_current_line_no = 0
    else:
          continue
    
    apps_df_current_datetime = apps_at_time_df.loc[apps_df_current_line_no,'timestamp_py']
    apps_at_time_df.loc[apps_df_current_line_no,'apps_started_this_hour'] += 1
    
    while ((apps_df_current_line_no < apps_at_time_df.shape[0] - 1) and\
            (apps_df_current_datetime < lsf_df_end_time)):

        apps_at_time_df.loc[apps_df_current_line_no,'apps_running'] += 1
        apps_at_time_df.loc[apps_df_current_line_no,'cores_avg'] += cores_used
        if (apps_at_time_df.loc[apps_df_current_line_no,'cores_min'] > cores_used):
            apps_at_time_df.loc[apps_df_current_line_no,'cores_min'] = cores_used
        if (apps_at_time_df.loc[apps_df_current_line_no,'cores_max'] < cores_used):
            apps_at_time_df.loc[apps_df_current_line_no,'cores_max'] = cores_used
            
        apps_df_current_line_no +=1
        apps_df_current_datetime = apps_at_time_df.loc[apps_df_current_line_no,'timestamp_py']
#     last row
    if (apps_df_current_datetime < lsf_df_end_time):
        apps_at_time_df.loc[apps_df_current_line_no,'apps_running'] += 1
        apps_at_time_df.loc[apps_df_current_line_no,'cores_avg'] += cores_used
        if (apps_at_time_df.loc[apps_df_current_line_no,'cores_min'] > cores_used):
            apps_at_time_df.loc[apps_df_current_line_no,'cores_min'] = cores_used
        if (apps_at_time_df.loc[apps_df_current_line_no,'cores_max'] < cores_used):
            apps_at_time_df.loc[apps_df_current_line_no,'cores_max'] = cores_used
    if (lsf_current_line['job status'].lower() == "done"):
        apps_at_time_df.loc[apps_df_current_line_no,'apps_ended_success_this_hour'] +=1
    else:
        apps_at_time_df.loc[apps_df_current_line_no,'apps_ended_exit_this_hour'] +=1
    apps_at_time_df.loc[apps_df_current_line_no,'apps_ended_this_hour'] +=1
            
    if (i % 1000 == 0):
        print(i)
apps_at_time_df['cores_sum'] = apps_at_time_df['cores_avg'].copy()
apps_at_time_df['cores_avg'] = apps_at_time_df['cores_avg']/apps_at_time_df['apps_running']

1000
2000
3000
4000
5000
6000


In [6]:
apps_at_time_df[:10]

,timestamp_py,date_py,time_py,apps_running,apps_running_ids,apps_started_this_hour,apps_ended_this_hour,apps_ended_success_this_hour,apps_ended_exit_this_hour,power_avg,power_min,power_max,energy_avg (kWh),energy_min (kWh),energy_max (kWh),cores_sum,cores_avg,cores_min,cores_max
0,2017-02-19 12:00:00,2017-02-19,12:00:00,31.0,0.0,31.0,0.0,0.0,0.0,40637,38600,41500,40.637000,38.600000,41.500000,15681.0,505.838710,1,1024
1,2017-02-19 13:00:00,2017-02-19,13:00:00,30.0,0.0,0.0,1.0,1.0,0.0,40863,40010,41530,40.863000,40.010000,41.530000,15680.0,522.666667,16,1024
2,2017-02-19 14:00:00,2017-02-19,14:00:00,30.0,0.0,0.0,0.0,0.0,0.0,39971,37740,41430,39.804454,37.582750,41.257375,15680.0,522.666667,16,1024
3,2017-02-19 15:00:00,2017-02-19,15:00:00,30.0,0.0,0.0,0.0,0.0,0.0,38517,36190,39470,38.677487,36.340792,39.634458,15680.0,522.666667,16,1024
4,2017-02-19 16:00:00,2017-02-19,16:00:00,32.0,0.0,2.0,0.0,0.0,0.0,38735,38330,39200,38.735000,38.330000,39.200000,15682.0,490.062500,1,1024
5,2017-02-19 17:00:00,2017-02-19,17:00:00,32.0,0.0,1.0,1.0,1.0,0.0,39358,35260,43690,39.358000,35.260000,43.690000,15682.0,490.062500,1,1024
6,2017-02-19 18:00:00,2017-02-19,18:00:00,35.0,0.0,6.0,3.0,3.0,0.0,41642,40900,42620,41.642000,40.900000,42.620000,16194.0,462.685714,1,1024
7,2017-02-19 19:00:00,2017-02-19,19:00:00,36.0,0.0,3.0,2.0,1.0,1.0,41635,39770,42620,41.635000,39.770000,42.620000,16305.0,452.916667,1,1024
8,2017-02-19 20:00:00,2017-02-19,20:00:00,37.0,0.0,3.0,2.0,2.0,0.0,42735,41130,45800,42.735000,41.130000,45.800000,16401.0,443.270270,1,1024
9,2017-02-19 21:00:00,2017-02-19,21:00:00,34.0,0.0,0.0,3.0,2.0,1.0,45315,44270,45860,45.315000,44.270000,45.860000,15409.0,453.205882,1,1024


------

## Weighted sum of applications running every second of hour.
weight = cores/max number of cores used by one app, i.e.1024

In [3]:
apps_each_sec = np.load('apps_each_sec.npy') 
apps_weighted_each_sec = np.load('apps_weighted_each_sec.npy') 

In [4]:
apps_each_sec = apps_each_sec[()]
apps_weighted_each_sec = apps_weighted_each_sec[()]

### Check if there are seconds when one or 0 applications are being run
\- No such moments

In [72]:
j=1
for k,v in apps_each_sec.items():
    if 0 in v.values():
        print('key of 0')
        print(list(v.keys())[list(v.values()).index(0)])
        print('key of 1')
        print(list(v.keys())[list(v.values()).index(1)])

In [6]:
apps_at_time_df['apps_weighted_max'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['apps_weighted_min'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df.head()

,timestamp_py,date_py,time_py,apps_running,apps_running_ids,apps_started_this_hour,apps_ended_this_hour,apps_ended_success_this_hour,apps_ended_exit_this_hour,power_avg,power_min,power_max,energy_avg (kWh),energy_min (kWh),energy_max (kWh),cores_sum,cores_avg,cores_min,cores_max,apps_weighted_max
0,2017-02-19 12:00:00,2017-02-19,12:00:00,31,0,31,0,0,0,40637,38600,41500,40.637000,38.600000,41.500000,15681,505.838710,1,1024,0.0
1,2017-02-19 13:00:00,2017-02-19,13:00:00,30,0,0,1,1,0,40863,40010,41530,40.863000,40.010000,41.530000,15680,522.666667,16,1024,0.0
2,2017-02-19 14:00:00,2017-02-19,14:00:00,30,0,0,0,0,0,39971,37740,41430,39.804454,37.582750,41.257375,15680,522.666667,16,1024,0.0
3,2017-02-19 15:00:00,2017-02-19,15:00:00,30,0,0,0,0,0,38517,36190,39470,38.677487,36.340792,39.634458,15680,522.666667,16,1024,0.0
4,2017-02-19 16:00:00,2017-02-19,16:00:00,32,0,2,0,0,0,38735,38330,39200,38.735000,38.330000,39.200000,15682,490.062500,1,1024,0.0


In [14]:
for i in range(0, zabbix_month_df.shape[0]):
    apps_at_time_df.at[i, 'apps_weighted_max'] = max(apps_weighted_each_sec[zabbix_month_df.timestamp_py.iloc[i]].values())
    apps_at_time_df.at[i, 'apps_weighted_min'] = min(apps_weighted_each_sec[zabbix_month_df.timestamp_py.iloc[i]].values())

In [16]:
apps_at_time_df.tail()

,timestamp_py,date_py,time_py,apps_running,apps_running_ids,apps_started_this_hour,apps_ended_this_hour,apps_ended_success_this_hour,apps_ended_exit_this_hour,power_avg,...,power_max,energy_avg (kWh),energy_min (kWh),energy_max (kWh),cores_sum,cores_avg,cores_min,cores_max,apps_weighted_max,apps_weighted_min
667,2017-03-19 07:00:00,2017-03-19,07:00:00,4,0,0,3,1,2,52631,...,53960,52.631,48.57,53.96,1312,328.0,64,704,1.28125,1.28125
668,2017-03-19 08:00:00,2017-03-19,08:00:00,4,0,0,0,0,0,52636,...,54140,52.636,48.19,54.14,1312,328.0,64,704,1.28125,1.28125
669,2017-03-19 09:00:00,2017-03-19,09:00:00,4,0,0,0,0,0,52713,...,54160,52.713,48.58,54.16,1312,328.0,64,704,1.28125,1.28125
670,2017-03-19 10:00:00,2017-03-19,10:00:00,4,0,0,0,0,0,50307,...,54030,50.307,38.77,54.03,1312,328.0,64,704,1.28125,1.28125
671,2017-03-19 11:00:00,2017-03-19,11:00:00,8,0,4,8,4,4,46478,...,50860,46.478,39.18,50.86,2304,288.0,64,800,2.06250,1.28125


In [17]:
apps_at_time_df['power_max_divide_by_weights'] = np.zeros(apps_at_time_df.shape[0])
apps_at_time_df['power_min_divide_by_weights'] = np.zeros(apps_at_time_df.shape[0])

In [21]:
apps_at_time_df['power_max_divide_by_weights'] = apps_at_time_df.power_max / (apps_at_time_df.apps_weighted_max * apps_at_time_df.cores_max.max() )
apps_at_time_df['power_min_divide_by_weights'] = apps_at_time_df.power_min /  (apps_at_time_df.apps_weighted_min * apps_at_time_df.cores_max.max())

In [23]:
apps_at_time_df.tail()

,timestamp_py,date_py,time_py,apps_running,apps_running_ids,apps_started_this_hour,apps_ended_this_hour,apps_ended_success_this_hour,apps_ended_exit_this_hour,power_avg,...,energy_min (kWh),energy_max (kWh),cores_sum,cores_avg,cores_min,cores_max,apps_weighted_max,apps_weighted_min,power_max_divide_by_weights,power_min_divide_by_weights
667,2017-03-19 07:00:00,2017-03-19,07:00:00,4,0,0,3,1,2,52631,...,48.57,53.96,1312,328.0,64,704,1.28125,1.28125,41.128049,37.019817
668,2017-03-19 08:00:00,2017-03-19,08:00:00,4,0,0,0,0,0,52636,...,48.19,54.14,1312,328.0,64,704,1.28125,1.28125,41.265244,36.730183
669,2017-03-19 09:00:00,2017-03-19,09:00:00,4,0,0,0,0,0,52713,...,48.58,54.16,1312,328.0,64,704,1.28125,1.28125,41.280488,37.027439
670,2017-03-19 10:00:00,2017-03-19,10:00:00,4,0,0,0,0,0,50307,...,38.77,54.03,1312,328.0,64,704,1.28125,1.28125,41.181402,29.550305
671,2017-03-19 11:00:00,2017-03-19,11:00:00,8,0,4,8,4,4,46478,...,39.18,50.86,2304,288.0,64,800,2.06250,1.28125,24.081439,29.862805


In [8]:
excel_writer_apps = pd.ExcelWriter( 'apps_at_time_py.xlsx')
apps_at_time_df.to_excel(excel_writer_apps)